In [1]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3

import random
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
#API_KEY = 'LbXORH3UxcFprtKYaDJWTtE8AbDXGMUm'
API_KEY = 'vhSXJ3LngQaN0WRhZkwqCEscsD7tWY7k'

In [15]:
def send_request(date):
    base_url = 'https://api.nytimes.com/svc/archive/v1'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    print(url)
    try:
        response = requests.get(url, verify=False).json()
    except Exception:
        return None
    time.sleep(10)
    return response


def is_valid(article, date):
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    is_US = article['section_name'] == 'U.S.'
    is_news = article['type_of_material'] == 'News'
    return is_in_range and has_headline and is_US and is_news


def parse_response(response): # we only need headline, date, web_url so I modified this part
    data = {'headline': [],
        'date': [],
        'web_url': [],
        #'doc_type': [],
        'lead_paragraph': [],
        #'material_type': [],
        #'author': [],
        #'section': [],
        #'subsection': [],
        #'keywords': []
            }
    #print(response)

    articles = response['response']['docs']
    for article in articles:
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main'])
            # if 'section_name' in article:
            #     data['section'].append(article['section_name'])
            # else:
            #     data['section'].append(None)
            if 'lead_paragraph' in article:
                data['lead_paragraph'].append(article['lead_paragraph'])
            else:
                data['lead_paragraph'].append(None)
            if 'web_url' in article:
                data['web_url'].append(article['web_url'])
            else:
                data['web_url'].append(None)
            # if 'subsection_name' in article:
            #     data['subsection'].append(article['subsection_name'])
            # else:
            #     data['subsection'].append(None)
            # if 'byline' in article:
            #     data['author'].append(article['byline']['original'])
            # else:
            #     data['author'].append(None)
            # data['doc_type'].append(article['document_type'])
            # if 'type_of_material' in article:
            #     data['material_type'].append(article['type_of_material'])
            # else:
            #     data['material_type'].append(None)
            # keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            # data['keywords'].append(keywords)
    # Combine the data into a list of tuples
    combined_data = list(zip(data['headline'], data['date'], data['web_url'], data['lead_paragraph']))

    # Sort the list based on the number of words in the 'lead_paragraph' field
    sorted_data = sorted(combined_data, key=lambda x: len(x[3].split()), reverse=True)

    # Select the top 100 entries
    if (len(sorted_data) < 50):
      sorted_data = sorted_data[:50]
    # n = len(data['headline'])
    # if (n < 100):
    #   return pd.DataFrame(data)
    # sample_indices = random.sample(range(n), 100)
    # sampled_data = {key: [data[key][i] for i in sample_indices] for key in data}

    return pd.DataFrame(sorted_data)


def get_data(dates):
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        print('Working on ' + str(date) + '...')
        csv_path = '/content/drive/MyDrive/Semester 6/5541 Team Folder/Project/data/nyt_headlines/' + date[0] + '-' + date[1] + '.csv'
        if not os.path.exists(csv_path): # If we don't already have this month
            response = send_request(date)
            if response is not None:
                df = parse_response(response)
                total += len(df)
                df.to_csv(csv_path, index=False)
                print('Saving ' + csv_path + '...')
    print('Number of articles collected: ' + str(total))

In [11]:
end = datetime.date(2024, 1, 1)
start = datetime.date(1998, 1, 1)
months = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]

In [ ]:
get_data(months)

Date range: ['1998', '1'] to ['2024', '1']
Working on ['1998', '1']...
https://api.nytimes.com/svc/archive/v1/1998/1.json?api-key=vhSXJ3LngQaN0WRhZkwqCEscsD7tWY7k
Saving /content/drive/MyDrive/Semester 6/5541 Team Folder/Project/data/nyt_headlines/1998-1.csv...
Working on ['1998', '2']...
https://api.nytimes.com/svc/archive/v1/1998/2.json?api-key=vhSXJ3LngQaN0WRhZkwqCEscsD7tWY7k
Saving /content/drive/MyDrive/Semester 6/5541 Team Folder/Project/data/nyt_headlines/1998-2.csv...
Working on ['1998', '3']...
https://api.nytimes.com/svc/archive/v1/1998/3.json?api-key=vhSXJ3LngQaN0WRhZkwqCEscsD7tWY7k
Saving /content/drive/MyDrive/Semester 6/5541 Team Folder/Project/data/nyt_headlines/1998-3.csv...
Working on ['1998', '4']...
https://api.nytimes.com/svc/archive/v1/1998/4.json?api-key=vhSXJ3LngQaN0WRhZkwqCEscsD7tWY7k
Saving /content/drive/MyDrive/Semester 6/5541 Team Folder/Project/data/nyt_headlines/1998-4.csv...
Working on ['1998', '5']...
https://api.nytimes.com/svc/archive/v1/1998/5.json?ap

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import pandas as pd
os.chdir("headlines") ## use Google Colab

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine in a single file
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

ValueError: No objects to concatenate